# JSON visualization routines

This is a set of visualization routines for the CyberAR project

In [5]:
import json
import ipywidgets as widgets
import os
import time
import numpy as np
import random
import pandas as pd

# Routines for json file creation and manipulations

The routines below are used to read, write, and concatenate json files for the app.  It allows us to use objects from disk as templates, combine modules/scenes, and output scenes.

In [139]:


# set up the helper routines to load the jsons
# written by J Wallin

def jsonFromFile(fname):
    f = open(fname,"r")
    s = f.read()
    f.close()
    return json.loads(s)

def jsonToFile(fname):
    print(fname)
    
def listFiles(assetPath, printFiles):
    finalList = []
    fullList = os.listdir(assetPath)
    for f in fullList:
        if f.find("json") == len(f)-4:
            print(f)
            finalList.append(f)
    return finalList

def loadFilesIntoDictionary(path, flist):
    jdata = {}
    for f in flist:
        jdata[f] = jsonFromFile(path + f)
    return jdata



def writeScene(myScene, verbose):
    # save it to an output file
    outputFile = myScene["jsonFileName"]

    print(os.path.isfile(outputPath + outputFile))

    # if there is a file already there
    print(outputPath + outputFile)
    fileExists = os.path.isfile(outputPath + outputFile)

    if verbose:
        while (fileExists or outputFile == "" ):
    
            print("This is a listing of files in this path: ")
            flist = os.listdir(outputPath)
            for f in flist:
                print(f)
            yn = input("The file " + outputPath + outputFile +" already exists \n Do you want to overwrite it? ")
            if yn == "y" or yn == "Y":
                fileExists = False
            else:
                outputFile = input("What is the new name for our file? ")
                fileExists = os.path.isfile(outputPath + outputFile)
                

    print("\nWriting " + outputPath+outputFile + "\n\n")


    # form the final scene as a string
    myScene["jsonFileName"] = outputFile
    finalJson = json.dumps(myScene, indent=4)
    f = open(outputPath + outputFile, "w", encoding='utf-8')
    f.write(json.dumps(myScene, ensure_ascii=False, indent=4))
    f.close()

    
def createScene(moduleDescription, sceneData, olist, verbose=False):
    sceneData["objects"] = olist
    
    
    # make a copy of the generic scene for the activity file
    myScene = jdata['genericActivity.json'].copy()

    # apply the module description data ot the sceneObject
    myScene.update(moduleDescription)
    myScene.update(sceneData)


    # we can dump the scene to see what it looks like so far  
    if verbose:
        print("This is the Scene\n\n\n")
        print(json.dumps(myScene, indent=4))    
    
    writeScene(myScene, verbose)    
    
    
# this is a code stub for merging two activity files together
def concatenateScenes( outputPath, outputFile, sceneList):

    print(outputPath, outputFile)
    # open the new file
    f = open(outputPath + outputFile, 'w', encoding='utf-8')
    
    for i in range(len(sceneList)):
        print("  \n scene")
        print(i)
        currentScene = sceneList[i]
        print(currentScene)
        
        # read in the files
        fn1 = outputPath + currentScene
        s1 = jsonFromFile(fn1)

        # Dump the file - but do NOT use any indents.  This will flatten
        # the file into a single line.  We then write it out as a line with a newline character.
        a1 = json.dumps(s1, ensure_ascii=False)
        f.write(a1 + "\n")

    # close it
    f.close()
    
    
  

# Object definition routines

These are helper routines to define objects and sets of objects commonly used in the app.  This includes routines to:

- Creating lines of flowing text using Text Mesh Pro
- Creating buttons with labels
- Modifying existing objects in a systematic way 


In [ ]:


# This routine creates a Text Mesh Pro text field.

def createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale):

    
    # this is a generic template - and will be overwritten
    tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }



    position1 = {"x" :topPosition['x'] + lineNumber * offSet['x'],
                "y": topPosition['y'] + lineNumber * offSet['y'],
                "z":topPosition['z'] + lineNumber * offSet['z']
               }

    tdata["textField"] = theText
    textObject = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": position1.copy(),
        "rotation": rotationAngle,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": parentObject,
        "name": textObjectName
    }
    
    return textObject.copy()





# Creates a butotn with specific size, label, location, angle and color.
# It returns a button and the associated label.

def createButton(name, 
                 label, 
                 position, 
                 labelOffset = 0.1,
                 scale = 1, 
                 parentObject = "[_DYNAMIC]",
                 color=[255,0,0,255], 
                 eulerAngles= {"x":90, "y":180.0, "z":0.0}):

    
    scriptDataA = {
    "name": "demoButtonActions",
    "callBackObjects": ["demoModule"]
    }
    scriptDataA = json.dumps(scriptDataA) 


    brbData = {
                "type": "Prefabs/demoButtonPrefab",
                "parentName": parentObject,
                "position": position,
                "eulerAngles": eulerAngles,
                "scale": {"x":scale, "y":scale, "z":scale},
                "name": name, 
                "color": color,
                "componentsToAdd": [scriptDataA]
            }
    
    labelPosition  = {"x":0.0, "y":0.0, "z":labelOffset}
    tdata = {
        "textField": "",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }
    tdata["textField"] = label
    labelScale = 0.6 * scale
    brbLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": labelPosition,
        "eulerAngles": eulerAngles,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": name,
        "name": name+"Label"  
    }
    return brbData, brbLabel



# This is a core routine for modifying an object in a clip.
# It helps set up the json based on inputs.

def createObjectModification(
    name, # object name
    activationCondition, 
    reactivateObject=False, 
    parentObject="[DYNAMIC]", 
    audioClipName = "",
    goCallback ="",
    newColor= None,
    enable=True, 
    newPosition=None, 
    newEulerAngles=None, 
    newScale=None,
    tmp=None):
    
    
    mm = {"name": name,
         "parentObject": parentObject,
         "activationConditions" : activationCondition,
         "reactiveObject": reactivateObject,
          "enable": enable,
          "position": {"x":0.0, "y":0.0, "z":0.0},
          "newPosition": False,
          "eulerAngles": {"x":0.0, "y":0.0, "z":0.0},
          "newEulerAngles": False,
          "scale": {"x":0.0, "y":0.0, "z":0.0},
          "newScale": False,
             }
                        
    if newPosition != None:
        mm["position"] = newPosition
        mm["newPosition"] = True
            
    if newEulerAngles != None:
        mm["eulerAngles"] = newEulerAngles
        mm["newEulerAngles"] = True
            
    if newScale != None:
        mm["scale"] = newScale
        mm["newScale"] = True
            
    if newColor != None:
        mm['color'] = newColor
    
    if tmp != None:
        mm['tmp'] = tmp
        
    return mm.copy()

# Functions to view and review schema from modules/scenes

The routines below translate the json into Excel files for review.  This is currently just a visualization 
technique so we can see the objects and the clips within a scene.

In [ ]:
######################################

# These are routines for changing the JSON in to pandas dataframes and excel
# files.  This allows us to view clips.


def flattenObjects(current):
    infoDictionary = {}
    columnData = ['name', 'parentName', 'position','rotation','scale','texture', 'childName','childColor','componentsToAdd']
    for c in columnData:
        infoDictionary[c] = []

    for o in current["objects"]:
        for c in columnData:    
            try:
                data = o[c]
            except:
                data = ""
            infoDictionary[c].append(data)
    # add the static objects
    


    for c in columnData:    
        if c == "name":
            s= "MainInstructions"
        else:
            s = ""
        infoDictionary[c].append(s)

        
    return infoDictionary


def flattenClips(current):
    
    
    clipDictionary = {}
    
    for c in clipInfo:
        clipDictionary[c] = []

    # get the basic information
    for o in current["clips"]:
        for c in clipInfo:    
            try:
                data = o[c]
            except:
                data = ""
            clipDictionary[c].append(data)
            
    return clipDictionary



def mapChanges(infoDictionary, clipDictionary):

    
    clipInfo = ['clipName', 'audioClipString', 'timeToEnd', 'autoAdvance', "changes"]
    objectMods1 = ['newPosition', 'newEulerAngles', "newScale"]

    objectNotation1 = ["P", "R", "S"]
    
    objectMods2 = ["canvasText", 'tmp', 'color']
    objectNotation2 = [ "C", "T", "c"]
    
    objectMods3 = ["componentsToAdd"]
    objectNotation3 = [ "x"]

    # get the list of all the objects in the scene
    objectList = infoDictionary['name']
    # form a matrix
    rows = len(objectList ) + 4  # 4 additional informaiton - clipname, timetoend, autoadvance, audio
    cols = len(current['clips']) + 1
    aa = np.empty((rows,cols), dtype=np.dtype('U100'))
    #aa[:,0] = ["clipName","timeToAdvance","autoAdvance","audioClipString"] + objectList
    aa[:,0] = ["clipName","timeToEnd","autoAdvance","audioClipString"] + objectList
    
    # loop through the clips
    for i in range(len(current['clips'])):
        
        # set the current clip name and column number/name
        col = i + 1
        clip = current['clips'][i]
        aa[0,col] = clip['clipName']
        aa[1,col] = clip['timeToEnd']
        aa[2,col] = clip['autoAdvance']
        try:
            ss = clip["audioClipString"]
        except:
            ss = ""
        aa[3,col] = ss
        # loop over the list of objects affected in this clip
        ochanges = clip['objectChanges']
        for oc in ochanges:    
            #print(clip['clipName'], len(ochanges))
            #print(" " + oc['name'])
            
            # using the object name, find the appropriate row
            # to update
            oname = oc['name']
            targetRow = -1
            for row in range(rows):
                val = aa[row,0]
                if (val == oname):
                    targetRow = row
                    aa[row,col] = "x"
            
            # make a string to note the types of object changes
            # the first set are boolean flags
            s = ":"
            for j in range(len(objectMods1)):
                m = objectMods1[j]
                try:
                    om = oc[m]
                    if om == True:
                        ss = objectNotation1[j]
                    else:
                        ss = ""
                except:
                    ss = ""
                s = s + ss
                
            # this is non-boolean values that need to be noted
            for j in range(len(objectMods2)):
                m = objectMods2[j]
                try:
                    om = oc[m]
                    ss = objectNotation2[j]
                except:
                    ss = ""
                s = s + ss
            
            # this is lists of objects that are not empty that need to be noted
            for j in range(len(objectMods3)):
                m = objectMods3[j]
                #print(m, oc[m])
                try:
                    om = oc[m]      
                    ss = len(om) * objectNotation3[j]
                except:
                    ss = ""
                s = s + ss
    
            if targetRow >= 0:
               aa[targetRow,col] = s

    return aa



# JSON comparision routines

These are routines for comparing two json objects.  You can use this to determine which parameters in a 
json has changed.  This can help track changes within objects.

These routines are not fully tested.

In [ ]:
    
def findKeyDiffs(baseClass, olist):
    # find the differences between two dictionaries
    
    # get the keys for the base class
    baseKeys = list(baseClass.keys())
    
    # find the set of common keys for the objects and the set of unique keys
    commonKeys = []
    uniqueKeys = []
    
    # get the key for each object
    for ii in range(len(olist)):
        cc = olist[i]
        objectKeys = list(cc.keys())
    
        # the union of the keys
        uKeys = list(set(baseKeys + objectKeys))
        
        # the intersection of keys
        iKeys = list(set(baseKeys).intersection(objectKeys))
        
        # find the elements only in one list or the other
        aKeys = list( set(baseKeys)^set(objectKeys))

        commonKeys = commonKeys + uKeys
        uniqueKeys = uniqueKeys + aKeys
        
    
    commonKeys = list(set(commonKeys))
    uniqueKeys = list(set(uniqueKeys))
    
    return commonKeys, uniqueKeys
    
        #for ii in range(len(k1)):
        #    k = k1[ii]
        #    if o1[k] != o2[k]:
        #        print(k)
    
    #


def findDiffs(commonKeys, baseClass, olist):
    
    # for each key
    for c in commonKeys:
        baseValue = baseClass[c]
        
        # are there any differences
        diffs = False
        for i in range(len(olist)):
            newValue = olist[i][c]
            if newValue != baseValue:
                diffs = True


        if diffs:
            print("{0:>15}".format(c), end="")
            for i in range(len(olist)):

                if baseValue != olist[i][c]:

                    print("  X   ", end="")
                else:
                    print("  .   ", end="")

            print()
    print(len(olist))
    
